<a href="https://colab.research.google.com/github/praveen61204/HCLTECH/blob/master/project/quality%20inspection/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install albumentations==1.3.0 opencv-python torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 2.0.8
    Uninstalling albumentations-2.0.8:
      Successfully uninstalled albumentations-2.0.8


In [1]:
import os
import cv2
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import models
import matplotlib.pyplot as plt

transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2(),
])


class DAGMClassificationDataset(Dataset):
    def __init__(self, img_dir, label_file, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        if not os.path.exists(label_file):
            print(f"Warning: Label file not found at {label_file}")
            return

        try:
            with open(label_file, "r") as f:
                lines = f.readlines()
            print(f"DEBUG: Read {len(lines)} lines from {label_file}")
        except Exception as e:
            print(f"Error reading label file {label_file}: {e}")
            return

        if not os.path.exists(img_dir):
            print(f"Warning: Image directory not found at {img_dir}")

        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 2:
                img_name = parts[0]
                label = int(parts[1])
                # Try common image extensions
                found_image_path = None
                for ext in ['.PNG', '.jpg', '.jpeg']:
                    img_path_with_ext = os.path.join(img_dir, img_name + ext)
                    if os.path.exists(img_path_with_ext):
                        found_image_path = img_path_with_ext
                        break

                if found_image_path:
                    self.image_paths.append(found_image_path)
                    self.labels.append(label)
                else:
                    print(f"Warning: Image not found: {os.path.join(img_dir, img_name)} (tried .png, .jpg, .jpeg)")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image=image)["image"]

        return image, torch.tensor(label, dtype=torch.long)

train_dir = "/content/drive/MyDrive/HCL/DAGM/Class10/Train/images"
train_labels = "/content/drive/MyDrive/HCL/DAGM/Class10/Train/Label/Labels.txt"

test_dir = "/content/drive/MyDrive/HCL/DAGM/Class10/Test/images"
test_labels = "/content/drive/MyDrive/HCL/DAGM/Class10/Test/Label/Labels.txt"

# --- Debugging additions ---
print(f"Checking existence of train_dir: {os.path.exists(train_dir)}")
print(f"Checking existence of train_labels file: {os.path.exists(train_labels)}")
print(f"Checking existence of test_dir: {os.path.exists(test_dir)}")
print(f"Checking existence of test_labels file: {os.path.exists(test_labels)}")

train_dataset = DAGMClassificationDataset(train_dir, train_labels, transform=transform)
test_dataset = DAGMClassificationDataset(test_dir, test_labels, transform=transform)

print(f"Number of samples in train_dataset: {len(train_dataset)}")
print(f"Number of samples in test_dataset: {len(test_dataset)}")

if len(train_dataset) == 0:
    raise ValueError("Train dataset is empty. Please check train_dir and train_labels paths and content.")
if len(test_dataset) == 0:
    raise ValueError("Test dataset is empty. Please check test_dir and test_labels paths and content.")
# --- End Debugging additions ---

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes: defect / non-defect
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"✅ Test Accuracy: {100 * correct / total:.2f}%")
torch.save(model.state_dict(), "dagm_resnet_classifier.pth")
print("Model saved successfully ✅")

Checking existence of train_dir: True
Checking existence of train_labels file: True
Checking existence of test_dir: True
Checking existence of test_labels file: True
DEBUG: Read 1151 lines from /content/drive/MyDrive/HCL/DAGM/Class10/Train/Label/Labels.txt
DEBUG: Read 1151 lines from /content/drive/MyDrive/HCL/DAGM/Class10/Test/Label/Labels.txt
Number of samples in train_dataset: 1150
Number of samples in test_dataset: 1150
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 150MB/s]


Epoch [1/5], Loss: 0.3221
Epoch [2/5], Loss: 0.0217
Epoch [3/5], Loss: 0.0225
Epoch [4/5], Loss: 0.0198
Epoch [5/5], Loss: 0.0138
✅ Test Accuracy: 99.91%
Model saved successfully ✅
